In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

import xarray as xr

import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model

from keras.models import load_model, Model, Sequential
from keras.layers import AveragePooling2D, MaxPooling2D, UpSampling2D,Conv2DTranspose, Input, SimpleRNN, LSTM, Dense,Dropout, Activation , Flatten, ConvLSTM2D, Conv2D, Reshape, TimeDistributed, BatchNormalization
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
from keras.utils.vis_utils import plot_model
from keras import regularizers
import keras

import tensorflow as tf


In [2]:
#Ubicación de los archivos

Data = r"/Users/javier/Documents/Data/"  #Ruta de acceso a los datos brutos 

Codigo = r'/Users/javier/Documents/Código/' #Dirección de códigos y funciones externas

Cache = r'/Users/javier/Documents/Código/Cache/' #Carpeta donde guardaremos los modelos y arreglos obtenidos

Imagenes = r'/Users/javier/Documents/Imagenes/' # Carpeta para guardar outputs gráficos


In [10]:
'''
Hace un avarage pooling y luego un flatten si se le pide.

'''


def preparar_x(dswind, lotes = 20000, pooling = 1, flat = False):

    '''
    Función para poder crear un arreglo de datos que contenga todos los dato del dataserie,
    
    Es relevante, ya que se debe hacer por lotes para no exceder la capacidad de la RAM.
    
    
    lotes: int, corresponde al tamaño de cada lote a procesar mediante un avarage pooling
    
    pooling: int, densidad del avarage pooling.
    
    return, arreglo X correspondiente a la base de datos dswind completa con dimensión reducida

    '''


    Largototal = dswind.data.shape[0]

    j = lotes

    X = resize(np.array(dswind.isel(time = slice(0,j)).data), pooling, pooling)

    
    for i in range(lotes*2,Largototal,lotes):
        
        windpart = resize(np.array(dswind.isel(time = slice(j,i)).data), pooling,pooling)
        
        X = np.append(X,windpart, axis = 0)
        
        j = i
        
        print('Datos: ',i,'/',Largototal)
        
    windpart = resize(np.array(dswind.isel(time = slice(j,Largototal)).data), pooling,pooling)

    X = np.append(X,windpart, axis = 0)
    
    if flat:
        
        X = X.reshape(X.shape[0],X.shape[1]*X.shape[2])
    
    return X

def RMSE(real, estimado):
    
    '''
    Raíz del error cuadrático medio'''
    
    return np.sqrt((np.square(real - estimado)).mean())




def resize(Wind,i,j=1,padding='valid'):
    
    
    '''
    Wind: array de dimensión (m, latitud, longitud) o  (m, latitud, longitud, canales)
    i: int, indica tamaño del pool size
    j: int, indica tamaño del strides 
    return: array de entrada con menor resolución. Dimensión (m, latitud/j, longitud/j, canales)
    '''
    
    if len(Wind.shape) ==3:
    
        Wind = Wind.reshape(Wind.shape[0], Wind.shape[1], Wind.shape[2],1)

    Average_pool = AveragePooling2D(pool_size = i,strides = j ,padding=padding)#, strides= int(10*i)) #el poolsize y stride, están pensados para las futuras iteraciones

    model = Sequential([Average_pool]) 

    output = model.predict(Wind)    
    
    Wind = np.squeeze(output) 
    
    return Wind


def flatten(Var):
    
    '''Funcion que realiza una transformación lineal de dimensión 3 a dimensión 2 '''
    
    return Var.reshape(Var.shape[0], Var.shape[1]*Var.shape[2])



def RMSE(real, estimado):
    
    '''
    Raíz del error cuadrático medio'''
    
    return np.sqrt((np.square(real - estimado)).mean())


def normalize(X):
    
    '''Normaliza los datos entre 1 y 0
    Es usada para el X de entrenamiento'''
    
    return (X-X.min())/(X.max()-X.min())


def juntar(Var):
    
    '''Luego de pronosticar con la red neuronl, se juntan las subseries, en una serie continua'''
    
    return Var.reshape(Var.shape[0]*Var.shape[1],Var.shape[2])


def dividir(Var1, Var2, timestep, desfase):
    
    '''Divide las series Var1 y Var2 en sub series de tamaño timestep más desfase
    esta funcion se usa dentro de get_set'''
    
    Var1_d = []

    for t in range(timestep+desfase, Var1.shape[0],timestep):
        
        Var1_d.append(Var1[t - timestep -desfase:t])
        
    Var1_d = np.array(Var1_d)

    Var2_d = []
    
    for t in range(timestep+desfase, Var2.shape[0],timestep):
            
        Var2_d.append(Var2[t - timestep:t])
        
    Var2_d = np.array(Var2_d)    
     
    return Var1_d, Var2_d

In [18]:


names = ['download(1979-1988).nc',
         'download(1989-1999).nc',
         'download(2000-2009).nc',
         ]

names2 = 'download(2010-2020).nc'


files = [Data + name for name in names]

dswind = xr.open_mfdataset(files)

dswind2 = xr.open_mfdataset(Data + names2).isel(expver=0)  

df0 = pd.read_csv(Data + 'NCEP_Spectra2.csv',index_col='Unnamed: 0')  #Sólo los parámetros de oleaje, no es el espectro completo.


df02 = pd.read_csv(Data + 'NCEP_Spectra_from_Partitions_33.0S_72W_197901to200912_paramsMIKE.csv',index_col='Unnamed: 0') 

In [20]:
#Escogemos el período de tiempo de interés y la ventana espacial que entregará las características necesarias para pronosticar.

Desde =  '2018-12-01 00:00:00'

Hasta =  '2019-02-01 03:00:00'

Longitud = slice(-130,-60)

Latitud = slice(-20,-70)


Presion = dswind2.msl.sel(time=slice(Desde,  Hasta),longitude=Longitud,latitude=Latitud)

#Pres = preparar_x(Presion,5000,2)  




In [24]:
Pres = preparar_x(Presion,490,2) 

In [25]:
Pres = normalize(Pres) #Normalizamos entre 0 y 1

In [26]:
#Cargar o guardar según corresponda el caso

#np.save('Presión_2018.npy',Pres)

Pres = np.load(Cache + 'Presión.npy')

In [10]:
#Repetimos tres veces la presión en un tercer canal para que los modelos pre-entrenados lo reciban como foto RGB

X3 = np.repeat(Pres[..., np.newaxis], 3, -1)



In [11]:
Pres.shape

(90575, 100, 140)

In [12]:
from tensorflow.keras.applications import InceptionV3, ResNet50, Xception,InceptionResNetV2, ResNet152V2, VGG16

input_tensor = Input(shape=(X3.shape[1], X3.shape[2], 3))

model =   VGG16(input_tensor=input_tensor , weights = 'imagenet', include_top =False)

model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 140, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 140, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 140, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 70, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 70, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 70, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 35, 128)       0     

In [55]:
#Hacemos el forward del modelo pre-entrenado, para obtener un vector x_features de características.
#puede tardar varios minutos

x_features = model.predict(X3)

np.save(Cache + 'x_features_VGG(2010-2020).npy', x_features)

print('Se guardó, ya tenemos el X de características')



Se guardó!


In [56]:
#Bajamos la dimensión y transformamos a vector.

X = resize(x_features,3,True)

X = flatten(X)

In [13]:
"'En caso de tener el x_features previamente guardado, se puede empezar aquí. Si no es el caso, saltar esta celda'"

X = np.load(Cache + 'x_features_VGG(1979-2009).npy')

X = resize(X,3, True)

X = flatten(X)

In [50]:
"'Se carga X como los valores de presión, omitir si se quiere usar el X features del modelo pre-entrenado'"

X = np.load(Cache + 'Presión2.npy')

X = resize(X,5,5)

X = flatten(X)

X.shape

(90575, 560)

## Regresor Ridge
Se usa el regresor para diferenciar los rendimientos de los distintos modelos pre-entrenados.

In [14]:

Desde = '1979-01-01 03:00:00'

Hasta = '2009-12-30 21:00:00'

Hm0 = np.array(df0[Desde: Hasta]['Hm0']).reshape(-1,1)
Tp = np.array(df0[Desde: Hasta]['Tp']).reshape(-1,1)
MD = np.array(df0[Desde: Hasta]['Mean Dir']).reshape(-1,1)
DSD =  np.array(df0[Desde: Hasta]['DSD']).reshape(-1,1)
WP = np.array(df02[Desde: Hasta]['P1: Wave Period, T01']).reshape(-1,1)

In [66]:
Y = MD # Seleccionamos el parámetro a pronosticar.

X2, Y = dividir(X, Y, timestep =1 , desfase=60) #Se prepara para ingresar al regresor por medio de un desfase.

X2 = flatten(X2)

Y= flatten(Y)


#Separamos en sets de entrenamiento y de validación

m = X2.shape[0]  

x_train_feat = X2[int(m*0.0):int(m*0.5)]

x_test_feat = X2[int(m*0.9):int(m*1)]

y_train_f = Y[int(m*0.0):int(m*0.5)]

y_test_f = Y[int(m*0.9):int(m*1)]


#Creamos el regresor

regresor = linear_model.Ridge(alpha=.5,normalize=True)

#Entrenamos el regresor

regresor = regresor.fit(x_train_feat,y_train_f)

In [64]:
m = X2.shape[0]  

x_train_feat = X2[int(m*0.0):int(m*0.5)]

x_test_feat = X2[int(m*0.9):int(m*1)]

y_train_f = Y[int(m*0.0):int(m*0.5)]

y_test_f = Y[int(m*0.9):int(m*1)]

In [67]:
#pronosticamos y calculamos r, rmse

y_pred_t = regresor.predict(x_test_feat)

print('r: ', np.round(np.corrcoef(y_test_f.T,y_pred_t.T)[0,1],3), '  RMSE: ',RMSE(y_test_f.T,y_pred_t.T))

r:  0.841   RMSE:  6.271389378271823
